In [15]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [16]:
import torch
import tensorflow as tf
import numpy as np

from time import time

pi = tf.constant(np.pi, dtype=tf.float32)

In [ ]:
def torch_fun_1(x):
    if not isinstance(x, torch.Tensor):
        raise ValueError(f"Input must be a pytorch tensor but is {type(x)}")
    if (2,) != x.shape:
        raise ValueError(f"Input must be of shape (2, ) but has shape {x.shape}")
    
    g = torch.Generator()
    g.manual_seed(42)
    y = x + torch.randint(4, 20, size=(2, ), generator=g)
    return sum(y**2)

def torch_fun_2(x):
    if not isinstance(x, torch.Tensor):
        raise ValueError(f"Input must be a pytorch tensor but is {type(x)}")
    if (10,) != x.shape:
        raise ValueError(f"Input must be of shape (10, ) but has shape {x.shape}")
    g = torch.Generator()
    g.manual_seed(42)
    y = x + torch.randint(4, 20, size=(10, ), generator=g)
    return sum(y**2)

def tf_fun_1(x):
    if not isinstance(x, tf.Variable):
        raise ValueError(f"Input must be a tensorflow Variable but is {type(x)}")
    if (2,) != x.shape:
        raise ValueError(f"Input must be of shape (2, ) but has shape {x.shape}")
    g = tf.random.Generator.from_seed(42)
    y = x + g.uniform(shape=(2,), minval=4, maxval=20, dtype=tf.float32)
    return sum(y**2)

def tf_fun_2(x):
    if not isinstance(x, tf.Variable):
        raise ValueError(f"Input must be a tensorflow Variable but is {type(x)}")
    if (10,) != x.shape:
        raise ValueError(f"Input must be of shape (10, ) but has shape {x.shape}")
    
    g = tf.random.Generator.from_seed(42)
    y = x + g.uniform(shape=(10,), minval=4, maxval=20, dtype=tf.float32)
    return sum(y**2)

In [18]:
def optimize_torch_fun1(f):
    """
    Finds arg min f

    Args:
        f: a function with torch operators only that receives a torch tensor of shape (2, ) and will evalue to a float

    Return: torch tensor of shape (2, )
    """
    pass

def optimize_torch_fun2(f):
    """
    Finds arg min f

    Args:
        f: a function with torch operators only that receives a torch tensor of shape (10, ) and will evalue to a float
    
    Return: torch tensor of shape (10, )
    """
    pass

def optimize_tf_fun1(f):
    """
    Finds arg min f

    Args:
        f: a function with tensorflow operators only that receives a tensorflow Variable of shape (2, ) and will evalue to a float
    
    Return: tensorflow Variable of shape (2, )
    """
    pass

def optimize_tf_fun2(f):
    """
    Finds arg min f

    Args:
        f: a function with tensorflow operators only that receives a tensorflow Variable of shape (10, ) and will evalue to a float
    
    Return: tensorflow Variable of shape (10, )
    """
    pass

In [19]:
def optimize_torch_fun1(f):
    """
    Encuentra arg min f utilizando PyTorch.

    Args:
        f: una función que recibe un tensor de PyTorch de forma (2,) y devuelve un float.

    Return: un tensor de PyTorch de forma (2,)
    """
    # 1. Inicializar el tensor 'x' que queremos optimizar.
    # Se le asigna requires_grad=True para que PyTorch rastree las operaciones y pueda calcular gradientes.
    x = torch.zeros(2, requires_grad=True)
    
    # 2. Definir el optimizador. Adam es una elección robusta y popular.
    # Le pasamos los parámetros a optimizar ([x]) y una tasa de aprendizaje (lr).
    optimizer = torch.optim.Adam([x], lr=0.1)
    
    # 3. Realizar los pasos de optimización en un bucle.
    for _ in range(1000):
        optimizer.zero_grad()  # Limpiar los gradientes de la iteración anterior.
        loss = f(x)            # Calcular el valor de la función (nuestra "pérdida").
        loss.backward()        # Calcular los gradientes de la pérdida con respecto a x.
        optimizer.step()       # Actualizar x en la dirección que minimiza la pérdida.
        
    return x

def optimize_torch_fun2(f):
    """
    Encuentra arg min f utilizando PyTorch.

    Args:
        f: una función que recibe un tensor de PyTorch de forma (10,) y devuelve un float.
    
    Return: un tensor de PyTorch de forma (10,)
    """
    x = torch.zeros(10, requires_grad=True)
    optimizer = torch.optim.Adam([x], lr=0.1)
    
    for _ in range(1000):
        optimizer.zero_grad()
        loss = f(x)
        loss.backward()
        optimizer.step()
        
    return x

def optimize_tf_fun1(f):
    """
    Encuentra arg min f utilizando TensorFlow.

    Args:
        f: una función que recibe una Variable de TensorFlow de forma (2,) y devuelve un float.
    
    Return: una Variable de TensorFlow de forma (2,)
    """
    # 1. Inicializar la variable 'x' que queremos optimizar.
    # tf.Variable es el tipo de dato para parámetros entrenables en TensorFlow.
    x = tf.Variable(tf.zeros((2,)), dtype=tf.float32)
    
    # 2. Definir el optimizador.
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
    
    # 3. Realizar los pasos de optimización en un bucle.
    for _ in range(1000):
        with tf.GradientTape() as tape: # Grabar las operaciones para el cálculo de gradientes.
            loss = f(x)                 # Calcular el valor de la función (pérdida).
        
        # Calcular los gradientes de la pérdida con respecto a x.
        gradients = tape.gradient(loss, [x])
        # Aplicar los gradientes para actualizar x.
        optimizer.apply_gradients(zip(gradients, [x]))

    return x

def optimize_tf_fun2(f):
    """
    Encuentra arg min f utilizando TensorFlow.

    Args:
        f: una función que recibe una Variable de TensorFlow de forma (10,) y devuelve un float.
    
    Return: una Variable de TensorFlow de forma (10,)
    """
    x = tf.Variable(tf.zeros((10,)), dtype=tf.float32)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

    for _ in range(1000):
        with tf.GradientTape() as tape:
            loss = f(x)
        
        gradients = tape.gradient(loss, [x])
        optimizer.apply_gradients(zip(gradients, [x]))

    return x

In [20]:
def score_fun(fun, optimization_fun, optimal_score, minimal_score_for_points, timeout):

    try:

        # run optimizer
        t_start = time()
        params = optimization_fun(fun)
        runtime = time() - t_start
        if runtime > timeout:
            print(f"Optimizer used {runtime}s, but only {timeout} are allowed")

        # determine performance
        funval = fun(params)
        if funval > minimal_score_for_points:
            return 0.0
        else:
            return np.round(float((100 * (minimal_score_for_points - funval) / (minimal_score_for_points - optimal_score))), 1)
        
    except Exception:
        raise

# run optimizer on first task
leaderboard_scores = []

scores = []
for name, obj_fun, optimizer_fun, least_possible_fun_val, fun_val_to_receive_points in [
    ("PyTorch Function 1", torch_fun_1, optimize_torch_fun1, 0, 100),
    ("PyTorch Function 2", torch_fun_2, optimize_torch_fun2, 0, 100),
    ("TensorFlow Function 1", tf_fun_1, optimize_tf_fun1, 0, 100),
    ("TensorFlow Function 2", tf_fun_2, optimize_tf_fun2, 0, 10)
]:
    score = score_fun(obj_fun, optimizer_fun, least_possible_fun_val, fun_val_to_receive_points, timeout=30)
    leaderboard_scores.append({
        "name": name,
        "value": score
    })
    scores.append(score)

In [21]:
print(scores)

[np.float64(100.0), np.float64(100.0), np.float64(100.0), np.float64(100.0)]
